# Global Overturning Streamfunction

This script is a test case for the OOD proof of concept testing.

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import cmocean as cm
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster

In [2]:
session = cc.database.create_session()

In [3]:
def plot_psi(psi_avg, clev=np.arange(-25,25,2)):

    p1 = plt.contourf(psi_avg.grid_yu_ocean, 
                 psi_avg.potrho, 
                 psi_avg, 
                 cmap=cm.cm.delta,levels=clev,extend='both')
    plt.contour(psi_avg.grid_yu_ocean, 
                psi_avg.potrho, 
                psi_avg, levels=clev, colors='k', linewidths=0.25)
    plt.contour(psi_avg.grid_yu_ocean,
                psi_avg.potrho, psi_avg,
                levels=[0.0,], colors='k', linewidths=0.5)
    plt.gca().invert_yaxis()
    
    plt.ylim((1037.5,1034))
    plt.ylabel('Potential Density (kg m$^{-3}$)')
    plt.xlabel('Latitude ($^\circ$N)')
    plt.xlim([-75,85])
    
    return p1

In [14]:
# OOD
cluster = SLURMCluster(cores=2,memory="31GB")
client = Client(cluster)
cluster.scale(cores=4)

In [40]:
client

Client Scheduler: tcp://10.0.96.2:36989 Dashboard: http://10.0.96.2:32971/status,Cluster Workers: 6 Cores: 6 Memory: 86.64 GiB


In [37]:
cluster.scale(cores=0)

In [ ]:
%%time
experiment = '01deg_jra55v13_ryf9091'
start_time='1950-01-01'
end_time='1999-12-31'
psi = cc.querying.getvar(experiment,'ty_trans_rho',session,start_time=start_time, end_time=end_time, frequency='1 monthly')
psi = psi.mean('time').sum('grid_xt_ocean')*1.0e-9
psi_avg = psi.cumsum('potrho') -  psi.sum('potrho')
psi_avg.load()

p1 = plot_psi(psi_avg)

distributed.core - ERROR - 'tcp://127.0.0.1:40471'
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/distributed/core.py", line 573, in handle_stream
    handler(**merge(extra, msg))
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/distributed/scheduler.py", line 5061, in handle_release_data
    ws: WorkerState = parent._workers_dv[worker]
KeyError: 'tcp://127.0.0.1:40471'
distributed.utils - ERROR - 'tcp://127.0.0.1:40471'
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/distributed/utils.py", line 671, in log_errors
    yield
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/distributed/scheduler.py", line 4073, in add_worker
    await self.handle_worker(comm=comm, worker=address)
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04

## Open On Demand 
| Workers | Cores | Memory | Number of years | CPU Time (s) | Wall Time (s) |  Notes | 
|----|----|----| ----|----|----|----|
| 0 | ? | ? | 1 | died | | Assume I don't have enough memory without a cluster |
| 4 | 8 | 28.88 Gb | 1 | 110 | 165 |  SLURMCluster(cores=8,memory="31GB")|
| 8 | 16 | 57.76 Gb | 1 | 97 | 129 |   |
| 8 | 16 | 57.76 Gb | 1 | 295 | 724 | New session. Seemed slow to start. Very slow to finish?? Not sure what is wrong here. |
| 8 | 16 | 57.76 Gb | 1 | 92 | 116 | Same session and cluster as above. Strange. Let's try again! |
| 8 | 16 | 57.76 Gb | 1 | 96 | 116 | Third time, just to check. |
| 8 | 32 | 57.76 Gb | 1 | 490 | 1554 | Trying SLURMCluster(cores=16,memory="31GB"). Very slow again...  |
| 16 | 16 | 115.52 GB | 1 | 96 | 120 | Hmm ... |
| 32 | 32 | 231.04 GB | 1 | 97 | 120 | OK. | 
| 32 | 32 | 231.04 GB | 5 |  |  | Gave up, for now. | 


## Gadi-Jupyter - normal queue
| Workers | Cores | Memory | # of years | CPU Time (s) | Wall Time (s) |  Notes | 
|----|----|----| ----|----|----|----|
| 8 | 16 | 64 Gb | 1 | 48 | 69 |   |
| 8 | 16 | 64 Gb | 1 | 47 | 57 | Second run just to check  |
| 8 | 16 | 64 Gb | 5 | 377 | 513 |  Slower than it should have been. Struggled with memory. |
| 8 | 16 | 64 Gb | 5 | 423 | 630 |  Checking. |
| 4 | 16 | 64 Gb | 10 | 1725 | 2282 |  Struggling! ... |
| 4 | 48 | 188.57 Gb | 10 | 586 | 767 |  Better! |
| 8 | 48 | 188.57 Gb | 10 | 294 | 388 |  Perhaps optimal? |
| 16 | 48 | 188.57 Gb | 10 | 417 | 487 | Looked pretty inefficient while running, but quicker. |
| 8 | 16 | 64 Gb | 50 | - | - | Ran out of memory  |
| 8 | 48 | 188.57 Gb | 50 | - | - |  Garbage collection warnings ... killed workers, etc.|
| 4 | 48 | 188.57 Gb | 50 | - | - |  |

We would like to be able to scale this calculation up to do 100 years at once. Gadi-jupyter can't do this, but we would like OOD to be able to.

## How does Slurm allocate cores and memory?

| SLURM cores | SLURM memory | cluster.scale(cores=?)  | client workers | client cores | client memory |
|---|---|---|---|---|---|
| 16 | 31 | 4 | 4 | 16 |28.88 | 
| 16 | 31 | 8 | 4 | 16 |28.88 | 
| 16 | 31 | 16 | 4 | 16 | 28.88 | 
| 16 | 31 | 32 | 8 | 32 | 57.76 | 
| 8 | 31 | 4 | 4 | 8 | 28.88 |
| 8 | 31 | 8 | 8 | 8 | 28.88 |
| 8 | 31 | 16 | 8 | 16 | 57.76 |
| 8 | 31 | 32 | 16 | 32 | 115.52 |
| 4 | 31 | 4 | 4 | 4 | 28.88 |
| 4 | 31 | 8 | 8 | 8 | 57.76 |
| 4 | 31 | 16 |16 | 16 | 115.52 |
| 4 | 31 | 32 | 32 | 32 | 231.04 |
| 2 | 31 | 4 | 4 | 4 | 57.76 |
| 2 | 31 | 8 | 6 | 6 | 86.64 |

I still find this strange -- but I think the way we should use this is:
* if you want multiple threads per worker, go for `SLURMCluster(cores=16,memory="31GB")`
* if you want lots of memory per worker, go for `SLURMCluster(cores=4,memory="31GB")`
* Then scale up cores, or number of workers via `cluster.scale()`
* As far as we can tell, `cluster.adapt()` doesn't work